In [1]:
import pandas as pd


In [2]:
# u.data
file_paths = {
    'u_data': './ml-100k/u.data',
    'u_info': './ml-100k/u.info',
    'u_item': './ml-100k/u.item',
    'u_genre': './ml-100k/u.genre',
    'u_user': './ml-100k/u.user',
    'u_occupation': './ml-100k/u.occupation',
}

In [ ]:
item

In [5]:
# 1. Load u.data
df = pd.read_csv(file_paths['u_data'], sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
df = df.drop(['timestamp'], axis=1)
df.head()

# 2. Load u.info
with open(file_paths['u_info'], 'r') as f:
  u_info = f.read().splitlines()

print("\nu_info List:")
print(u_info)


# 3. Load u.item
item_df = pd.read_csv(file_paths['u_item'], sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                            'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
item_df = item_df.drop(['video_release_date', 'IMDb_URL'], axis=1)
item_df.head()


# 4. movie_dict
movie_dict = dict(zip(item_df['movie_id'], item_df['movie_title']))
print({k: movie_dict[k] for k in list(movie_dict)[:5]})

# 4. Load u.genre
genre_df = pd.read_csv(file_paths['u_genre'], sep='|', header=None, names=['genre', 'genre_id'])
genre_df.head()

# 5. Load u.user
user_df = pd.read_csv(file_paths['u_user'], sep='|', header=None, names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
user_df.head()


# 6. Load u.occupation
occupation_df = pd.read_csv(file_paths['u_occupation'], header=None, names=['occupation'])
occupation_df.head()


item_df.rename(columns={"movie_id":"item_id"}, inplace=True)


u_info List:
['943 users', '1682 items', '100000 ratings']
{1: 'Toy Story (1995)', 2: 'GoldenEye (1995)', 3: 'Four Rooms (1995)', 4: 'Get Shorty (1995)', 5: 'Copycat (1995)'}


In [15]:
df  # 사용자별로 아이템에 점수를 매긴 데이터
item_df # 영화에 대한 종류 (제목, 개봉날짜, 장르) (one-hot encoding)
movie_dict # 영화 id와 제목에 대한 mapping 정보
genre_df # 장르 id와 장르명에 대한 mapping 정보 
user_df # 사용자id별  나이, 성별, 직업에 대한 정보

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [6]:
merged_df = pd.merge(df, item_df, on="item_id", how="left")
merged_df = pd.merge(merged_df, user_df, on="user_id", how="left")
# merged_df.to_csv("./ml_merged.csv")

In [7]:
merged_df

,user_id,item_id,rating,movie_title,release_date,unknown,Action,Adventure,Animation,Children's,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,gender,occupation,zip_code
0,196,242,3,Kolya (1996),24-Jan-1997,0,0,0,0,0,...,0,0,0,0,0,0,49,M,writer,55105
1,186,302,3,L.A. Confidential (1997),01-Jan-1997,0,0,0,0,0,...,1,0,0,1,0,0,39,F,executive,00000
2,22,377,1,Heavyweights (1994),01-Jan-1994,0,0,0,0,1,...,0,0,0,0,0,0,25,M,writer,40206
3,244,51,2,Legends of the Fall (1994),01-Jan-1994,0,0,0,0,0,...,0,1,0,0,1,1,28,M,technician,80525
4,166,346,1,Jackie Brown (1997),01-Jan-1997,0,0,0,0,0,...,0,0,0,0,0,0,47,M,educator,55113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,"First Wives Club, The (1996)",14-Sep-1996,0,0,0,0,0,...,0,0,0,0,0,0,13,M,student,83702
99996,716,204,5,Back to the Future (1985),01-Jan-1985,0,0,0,0,0,...,0,0,1,0,0,0,36,F,administrator,44265
99997,276,1090,1,Sliver (1993),01-Jan-1993,0,0,0,0,0,...,0,0,0,1,0,0,21,M,student,95064
99998,13,225,2,101 Dalmatians (1996),27-Nov-1996,0,0,0,0,1,...,0,0,0,0,0,0,47,M,educator,29206


In [11]:
# Create user-item matrix --> dummy_data.csv
user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='rating')
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
